# Problem Statement
- The U.S. has almost 500 students for every guidance counselor. Underserved youth lack the network to find their career role models, making CareerVillage.org the only option for millions of young people in America and around the globe with nowhere else to turn.

- To date, 25,000 volunteers have created profiles and opted in to receive emails when a career question is a good fit for them. This is where your skills come in. To help students get the advice they need, the team at CareerVillage.org needs to be able to send the right questions to the right volunteers. The notifications sent to volunteers seem to have the greatest impact on how many questions are answered.

# Objective: develop a method to recommend relevant questions to the professionals who are most likely to answer them.

In [2]:
import nltk, urllib, request, string, re
from nltk import word_tokenize
from nltk.corpus import stopwords
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
import os
os.chdir('D:\\siuol456\\Documents\\NEU\\Study group 2019 Summer\\FinalProject\\datasets')

In [37]:
Q_body=pd.read_csv('questions.csv')
Q_body.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a ma...
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,To become a specialist in business management...,i hear business management is a hard way to ge...
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...


In [94]:
def cleansing(s,n):
    raw = BeautifulSoup(s, 'html.parser').get_text()
    raw=raw.lower()
    raw=re.sub(r'\d+', '', raw)
    tokens = word_tokenize(raw)
    sw = stopwords.words('English')
    new_token=list()
    for t in tokens:
        if t not in sw:
            new_token.append(t)
    punc=list(string.punctuation)
    punc.append('“')
    punc.append('”')
    punc.append('’')
    punc.append("n't")
    punc.append("'m")
    punc.append("th")
    punc.append("'d")
    new_token2=list()
    for t in new_token:
        if t not in punc:
            new_token2.append(t)
    freq= nltk.FreqDist(new_token2)
    tryo = pd.DataFrame(list(dict(freq).items()))
    tryo=tryo.sort_values(by=1, ascending=False)
    tryo.columns = ['Words', 'freq']
    df=tryo[0:n]
    result=list(df['Words'])
    return(result)

In [101]:
cleansing(Q_body.questions_body.iloc[654],5)

['classes', 'class', 'electives', 'engineering', 'high']

In [88]:
Q_body.questions_body[Q_body.questions_id=='72f843e027b24a1ea621b90d479d9db9']

1422    What steps should I take as an undergrad to pr...
Name: questions_body, dtype: object

In [100]:
Q_body.questions_body.iloc[654]

'In high school, classes are separated into core classes (main subjects like Mathematics, English, Science, etc.) and electives (classes outside a required curriculum like Creative Writing, Band, Ceramics, etc.). Electives may not seem that important because they are more of a "just because" class, so many take classes that are laid-back and don\'t require much work and energy. On the other hand, elective courses also allow you to explore your passions, interests, and career opportunities. I have taken both career-based and laid-back elective courses before and I have to say, both have their positives and negatives. It is hard for me to pick a career-related course though because I don\'t know if I really want to waste a period on a class that I could take up in college if I wanted to. I took an Engineering Drawing course my freshman year of high school due to an interest in engineering I had then. By the end of the year, I wanted nothing to do with engineering. When looking back, I kn